<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/neqsimreaktoro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use of third party libraries in NeqSim process simulations
In this workbook we will give examples of how to use third party tools such as Reaktoro and Refprop in a NeqSim process simulation.

# Use of Reaktoro as a unit operation in NeqSim process simulation
In the following example we will integrate Reaktoro as an unit operation in a NeqSim process simulation

In [1]:
!pip install neqsim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 16.8 MB/s eta 0:00:00


In [2]:
%%capture
!pip install -q condacolab
import condacolab
condacolab.install_from_url("https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh")
!conda config --remove channels defaults
!conda config --add channels conda-forge
!conda install reaktoro -y

# Implementing a Reaktoro unit operation in NeqSim

In [1]:
from neqsim.process.unitop import unitop
from neqsim import jNeqSim
from jpype import JImplements, JOverride
import reaktoro as rkt
import json

class ReaktoroUnitOperation(unitop):
    def __init__(self):
        super().__init__()
        self.name = ""
        self.inputstream = None
        self.aprops = None
        self.fugacity_co2 = None

    def setInputStream(self, stream):
        self.inputstream = stream

    def calc_reaktoro(self, pressure, temperature , fugacity_co2):
      db = rkt.PhreeqcDatabase("phreeqc.dat")
      solution = rkt.AqueousPhase(rkt.speciate("H O Na Cl C Ca"))
      solution.set(rkt.ActivityModelPhreeqc(db))

      calcite = rkt.MineralPhase("Calcite")

      system = rkt.ChemicalSystem(db, solution, calcite)

      specs = rkt.EquilibriumSpecs(system)
      specs.temperature()
      specs.pressure()
      specs.fugacity("CO2(g)")

      solver = rkt.EquilibriumSolver(specs)

      state = rkt.ChemicalState(system)
      state.temperature(50.0, "celsius")
      state.pressure(10.0, "bar")
      state.set("H2O", 1.0, "kg")
      state.set("Na+", 1.0, "mol")
      state.set("Cl-", 1.0, "mol")
      state.set("Calcite", 10, "g")

      conditions = rkt.EquilibriumConditions(specs)
      conditions.temperature(temperature, "celsius")
      conditions.pressure(pressure, "bar")
      conditions.fugacity("CO2(g)", fugacity_co2, "bar")

      result = solver.solve(state, conditions)
      return rkt.AqueousProps(state)

    @JOverride
    def run(self):
      print("ReaktoroUnitOperation.run()")
      self.fugacity_co2 = self.inputstream.getFluid().getPhase(0).getFugacity('CO2')
      self.aprops = self.calc_reaktoro(self.inputstream.getPressure('bara'), self.inputstream.getTemperature('C'), self.fugacity_co2)

    @JOverride
    def toJson(self):
      data_dict = {
            "name": self.name,
            "pH": self.aprops.pH()[0],
            "fugacity CO2": self.fugacity_co2
      }
      return json.dumps(data_dict)


# Implement the Reaktoro unit operation in a NeqSim simulation
The unit operation will report the pH of an aqueous phase with some ions in equilibrium with the gas.

In [2]:
from neqsim.thermo import fluid
fluid1 = fluid("srk")  # create a fluid using the SRK-EoS
fluid1.setTemperature(30.0, "C")
fluid1.setPressure(100.0, "bara")
fluid1.addComponent("methane", 0.9)
fluid1.addComponent("CO2", 0.1)
fluid1.setMixingRule(2)

stream1 = jNeqSim.processSimulation.processEquipment.stream.Stream(fluid1)
stream1.setFlowRate(30000, "kg/hr")

uop = ReaktoroUnitOperation()
uop.setName("Reaktoro Unit Operation")
uop.setInputStream(stream1)

example_process = jNeqSim.processSimulation.processSystem.ProcessSystem()
example_process.add(stream1)
example_process.add(uop)

example_process.run()


In [3]:
uop.run() # this should run when process run - but seems not to
json_report= str(jNeqSim.processSimulation.util.report.Report(example_process).generateJsonReport())
output = json.loads(json_report)
print(output['Reaktoro Unit Operation'])

ReaktoroUnitOperation.run()
{'name': 'Reaktoro Unit Operation', 'pH': 5.520539435475417, 'fugacity CO2': 6.970782669382442}


# Use of CoolProp as a unit operation in NeqSim process simulation
In the following example we will integrate CoolProp as an unit operation in a NeqSim process simulation

http://www.coolprop.org/coolprop/examples.html

In [4]:
!pip install CoolProp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.9 MB/s eta 0:00:00


In [31]:
from neqsim.process.unitop import unitop
from neqsim import jNeqSim
from jpype import JImplements, JOverride
import CoolProp.CoolProp as CP
from CoolProp.CoolProp import PropsSI
import json

class CoolPropUnitOperation(unitop):
    def __init__(self):
        super().__init__()
        self.name = ""
        self.inputstream = None
        self.densityCO2 = None
        self.densityCO2_neqsim = None

    def setInputStream(self, stream):
        self.inputstream = stream

    def calc_cool_prop(self):
      return PropsSI('D', 'T', self.inputstream.getTemperature('K'), 'P', self.inputstream.getPressure('Pa'), 'CO2')

    @JOverride
    def run(self):
      self.inputstream.getFluid().initPhysicalProperties('Density')
      self.densityCO2 = self.calc_cool_prop()
      self.densityCO2_neqsim = self.inputstream.getFluid().getPhase(0).getDensity('kg/m3')

    @JOverride
    def toJson(self):
      data_dict = {
            "name": self.name,
            "densityCO2_CoolProp": self.densityCO2,
            "densityCO2_NeqSim": self.densityCO2_neqsim
      }
      return json.dumps(data_dict)

In [32]:
from neqsim.thermo import fluid
fluid1 = fluid("srk")  # create a fluid using the SRK-EoS
fluid1.setTemperature(14.3, "C")
fluid1.setPressure(55.0, "bara")
fluid1.addComponent("CO2", 0.1)
fluid1.setMixingRule(2)

stream1 = jNeqSim.processSimulation.processEquipment.stream.Stream(fluid1)
stream1.setFlowRate(30000, "kg/hr")

uop = CoolPropUnitOperation()
uop.setName("CoolProp Unit Operation")
uop.setInputStream(stream1)

example_process = jNeqSim.processSimulation.processSystem.ProcessSystem()
example_process.add(stream1)
example_process.add(uop)

example_process.run()

In [33]:
uop.run() # this should run when process run - but seems not to
json_report= str(jNeqSim.processSimulation.util.report.Report(example_process).generateJsonReport())
output = json.loads(json_report)
print(output['CoolProp Unit Operation'])

{'name': 'CoolProp Unit Operation', 'densityCO2_CoolProp': 837.1416548396195, 'densityCO2_NeqSim': 742.8594991083878}


# Use of ThermoPack as a unit operation in NeqSim process simulation
In the following example we will integrate ThermoPack as an unit operation in a NeqSim process simulation

https://pypi.org/project/thermopack/

In [8]:
!pip install thermopack

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 23.3 MB/s eta 0:00:00


In [28]:
from neqsim.process.unitop import unitop
from neqsim import jNeqSim
from jpype import JImplements, JOverride
from thermopack.saftvrmie import saftvrmie
import json

class ThermoPackUnitOperation(unitop):
    def __init__(self):
        super().__init__()
        self.name = ""
        self.inputstream = None
        self.densityCO2 = None
        self.densityCO2_neqsim = None

    def setInputStream(self, stream):
        self.inputstream = stream

    def calc_thermo_pack_prop(self):
      eos = saftvrmie('C1,CO2')
      T = 300 # Kelvin
      p = 1e5 # Pascal
      x = self.inputstream.getFluid().getzvector() # Molar composition
      vg, = eos.specific_volume(self.inputstream.getTemperature('K'), self.inputstream.getPressure('Pa'), x, eos.TWOPH) # Molar volume of gas phase (NB: Notice the comma)
      return self.inputstream.getFluid().getPhase(0).getMolarMass('kg/mol')/vg

    @JOverride
    def run(self):
      self.inputstream.getFluid().initPhysicalProperties('Density')
      self.densityCO2 = self.calc_thermo_pack_prop()
      self.densityCO2_neqsim = self.inputstream.getFluid().getPhase(0).getDensity('kg/m3')

    @JOverride
    def toJson(self):
      data_dict = {
            "name": self.name,
            "densityCO2_ThermoPack": self.densityCO2,
            "densityCO2_NeqSim": self.densityCO2_neqsim
      }
      return json.dumps(data_dict)

In [29]:
from neqsim.thermo import fluid
fluid1 = fluid("srk")  # create a fluid using the SRK-EoS
fluid1.setTemperature(14.3, "C")
fluid1.setPressure(65.0, "bara")
fluid1.addComponent("methane", 0.01)
fluid1.addComponent("CO2", 0.99)
fluid1.setMixingRule(2)

stream1 = jNeqSim.processSimulation.processEquipment.stream.Stream(fluid1)
stream1.setFlowRate(30000, "kg/hr")

uop = ThermoPackUnitOperation()
uop.setName("ThermoPack Unit Operation")
uop.setInputStream(stream1)

example_process = jNeqSim.processSimulation.processSystem.ProcessSystem()
example_process.add(stream1)
example_process.add(uop)

example_process.run()

In [30]:
uop.run() # this should run when process run - but seems not to
json_report= str(jNeqSim.processSimulation.util.report.Report(example_process).generateJsonReport())
output = json.loads(json_report)
print(output['ThermoPack Unit Operation'])

{'name': 'ThermoPack Unit Operation', 'densityCO2_ThermoPack': 856.638171127874, 'densityCO2_NeqSim': 751.3317350536282}
